# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [1]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [3]:
# Constants for temperature conversion
FAHRENHEIT_TO_CELSIUS_MULTIPLIER = 5 / 9
FAHRENHEIT_TO_CELSIUS_OFFSET = 32

def read_temperatures_from_csv(input_file):
    """Reads temperatures in Fahrenheit from a CSV file and returns them as a list of floats."""
    temperatures_fahrenheit = []
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            temperatures_fahrenheit.append(float(row[0]))
    return temperatures_fahrenheit

def convert_to_celsius(temperatures_fahrenheit):
    """Converts a list of Fahrenheit temperatures to Celsius."""
    return [
        (temp - FAHRENHEIT_TO_CELSIUS_OFFSET) * FAHRENHEIT_TO_CELSIUS_MULTIPLIER
        for temp in temperatures_fahrenheit
    ]

def calculate_statistics(temperatures_celsius):
    """Calculates and returns the average, minimum, and maximum of a list of temperatures."""
    average = sum(temperatures_celsius) / len(temperatures_celsius)
    minimum = min(temperatures_celsius)
    maximum = max(temperatures_celsius)
    return average, minimum, maximum

def write_temperatures_to_csv(output_file, temperatures_celsius):
    """Writes a list of Celsius temperatures to a CSV file."""
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])
        for temp in temperatures_celsius:
            writer.writerow([temp])

def process_temperature_data(input_file, output_file):
    """Main function to read, process, and save temperature data."""
    # Step 1: Read temperatures from the input file
    temperatures_fahrenheit = read_temperatures_from_csv(input_file)

    # Step 2: Convert temperatures to Celsius
    temperatures_celsius = convert_to_celsius(temperatures_fahrenheit)

    # Step 3: Calculate and print statistics
    average, minimum, maximum = calculate_statistics(temperatures_celsius)
    print("Statistics:")
    print(f"Average: {average:.2f}°C")
    print(f"Minimum: {minimum:.2f}°C")
    print(f"Maximum: {maximum:.2f}°C")

    # Step 4: Write temperatures to the output file
    write_temperatures_to_csv(output_file, temperatures_celsius)


### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [2]:
from unittest.mock import mock_open, patch

class UnitTestCase(unittest.TestCase):
    def test_read_temperatures_from_csv(self):
        # Mock CSV content
        mock_csv = "Temperature (F)\n78.5\n81.2\n75.9\n82.1\n"
        with patch("builtins.open", mock_open(read_data=mock_csv)):
            result = read_temperatures_from_csv("input.csv")
        self.assertEqual(result, [78.5, 81.2, 75.9, 82.1])

    def test_convert_to_celsius(self):
        temperatures_fahrenheit = [78.5, 81.2, 75.9, 82.1]
        result = convert_to_celsius(temperatures_fahrenheit)
        expected = [(78.5 - 32) * 5 / 9, (81.2 - 32) * 5 / 9, (75.9 - 32) * 5 / 9, (82.1 - 32) * 5 / 9]
        self.assertAlmostEqual(result, expected, places=2)

    def test_calculate_statistics(self):
        temperatures_celsius = [25.0, 27.0, 23.0, 28.0]
        average, minimum, maximum = calculate_statistics(temperatures_celsius)
        self.assertAlmostEqual(average, 25.75, places=2)
        self.assertEqual(minimum, 23.0)
        self.assertEqual(maximum, 28.0)

    def test_write_temperatures_to_csv(self):
        # Prepare mock data
        temperatures_celsius = [25.0, 27.0, 23.0, 28.0]
        mock_file = mock_open()

        with patch("builtins.open", mock_file):
            write_temperatures_to_csv("output.csv", temperatures_celsius)

        # Validate the written data
        mock_file().write.assert_any_call("Temperature (C)\n")
        for temp in temperatures_celsius:
            mock_file().write.assert_any_call(f"{temp}\n")

class IntegrationTestCase(unittest.TestCase):
    @patch("builtins.open", new_callable=mock_open)
    def test_process_temperature_data(self, mock_file):
        # Mock input CSV content
        mock_file.side_effect = [
            mock_open(read_data="Temperature (F)\n78.5\n81.2\n75.9\n82.1\n").return_value,
            mock_open().return_value,
        ]

        # Run the main function
        process_temperature_data("input.csv", "output.csv")

        # Validate the output CSV
        mock_file().write.assert_any_call("Temperature (C)\n")
        mock_file().write.assert_any_call("25.83\n")  # 78.5 converted to Celsius
        mock_file().write.assert_any_call("27.33\n")  # 81.2 converted to Celsius